In [4]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn
from torchvision.models import resnet18

from sklearn.model_selection import train_test_split

In [21]:
# Set up data split
image_paths = pd.read_csv('image_paths.csv')

# Split data into training and testing
train_paths, test_paths = train_test_split(image_paths, test_size=0.2)

In [ ]:
# Load the model
model = resnet18(pretrained=True)
model.fc = nn.Linear(512, 2)

# Load the data
def load_image(path):
    return plt.imread(path)

def load_data(paths):
    images = np.array([load_image(path) for path in paths])
    images = torch.tensor(images, dtype=torch.float32)
    return images

train_images = load_data(train_paths['path'])
train_labels = torch.tensor(train_paths['label'].values, dtype=torch.long)

test_images = load_data(test_paths['path'])
test_labels = torch.tensor(test_paths['label'].values, dtype=torch.long)

# Train the model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(train_images)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch} loss: {loss.item()}')

# Test the model
with torch.no_grad():
    outputs = model(test_images)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == test_labels).sum().item()
    total = test_labels.size(0)
    print(f'Accuracy: {correct / total * 100}%')

# Save the model
torch.save(model.state_dict(), 'model.pth')